In [ ]:
import sys 
sys.path.insert(1, "..")
from paseos import ActorBuilder, SpacecraftActor
import pykep as pk
import numpy as np
import paseos
from paseos.visualization.space_animation import SpaceAnimation
from paseos.utils.set_log_level import set_log_level
from paseos.utils.load_default_cfg import load_default_cfg
set_log_level("INFO")
%load_ext autoreload
%autoreload 2

In [ ]:
# Define central body
earth = pk.planet.jpl_lp("earth")
#Define today (27-10-22)
today = pk.epoch(8335.5,"mjd2000")

## 1) - Instantiate Sentinel2-A and Sentinel2-B space actors.

First, we can build the `scaffold`for  **Sentinel2-A** (S2A) and **Sentinel2-B** (S2B).

In [ ]:
# Define local actor
S2A = ActorBuilder.get_actor_scaffold(
    "Sentinel2-A", SpacecraftActor, [0,0,0], today
)

# Define local actor
S2B = ActorBuilder.get_actor_scaffold(
    "Sentinel2-B", SpacecraftActor, [0,0,0], today
)

The next step is adding an orbit. To this aim it is possible to use  `Two-Lines Elements (TLEs)` values (27-10-2022) to calculate **Sentinel2-A** and **Sentinel2-B** ephemerides.

In [ ]:
sentinel2A_line1 = "1 40697U 15028A   22300.01175178 -.00001065  00000+0 -38995-3 0  9999"
sentinel2A_line2 = "2 40697  98.5650  12.8880 0001080  78.8662 281.2690 14.30806819383668"
sentinel2A = pk.planet.tle(sentinel2A_line1, sentinel2A_line2)

#Calculating S2A ephemerides.
sentinel2A_eph=sentinel2A.eph(today)

sentinel2B_line1 = "1 42063U 17013A   22300.18652110  .00000099  00000+0  54271-4 0  9998"
sentinel2B_line2 = "2 42063  98.5693  13.0364 0001083 104.3232 255.8080 14.30819357294601"
sentinel2B = pk.planet.tle(sentinel2B_line1, sentinel2B_line2)

#Calculating S2B ephemerides.
sentinel2B_eph=sentinel2B.eph(today)

Now it is possible to use the ephemerides values to build the S2A and S2B `space actors`.

In [ ]:
#Adding orbits around Earth based on previously calculated ephemerides
ActorBuilder.set_orbit(S2A, sentinel2A_eph[0], sentinel2A_eph[1], today, earth)
ActorBuilder.set_orbit(S2B, sentinel2B_eph[0], sentinel2B_eph[1], today, earth)

It is now possible to add `Power devices` and `Communication devices` for both the satellites. In this case, we assume `S2A` ans `S2B` to be equipped with an `Inter-Satellite-Link (ISL)` to enable their communication. <br> Since in this simulation `S2A` is the `local_actor`, we add these devices only for it.

In [ ]:
#Adding power devices
ActorBuilder.set_power_devices(S2A, 500, 10000, 1)
#Adding communication devices.
ActorBuilder.add_comm_device(S2A, "isl_transmitter", 10000)

Plotting `S2A` and `S2B` in their orbit around Earth. As you can see, `S2A` and `S2B` are not in line of sight and they are in opposite phase inside the orbit to maximize the Earth coverage. Despite that, this will prevent them from communicating.

In [ ]:
#Creating Earth model
u, v = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
x = pk.EARTH_RADIUS * np.cos(u) * np.sin(v)
y = pk.EARTH_RADIUS * np.sin(u) * np.sin(v)
z = pk.EARTH_RADIUS * np.cos(v)
# Plotting S2A and S2B orbit around Earth.
import matplotlib.pyplot as plt
ax=pk.orbit_plots.plot_planet(S2A._orbital_parameters, today, color="r",legend=("S2A",""))
pk.orbit_plots.plot_planet(S2B._orbital_parameters, today,color="b",legend=("S2B",""), axes=ax)
ax.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r, alpha=0.4)

## 2) - Initialize PASEOS simulation

In [ ]:
# init simulation
cfg=load_default_cfg()
cfg.sim.start_time=today.mjd2000 * pk.DAY2SEC
sim = paseos.init_sim(S2A, cfg)

In [ ]:
sim.add_known_actor(S2B)

In [ ]:
anim = SpaceAnimation(sim)

In [ ]:
anim = SpaceAnimation(sim)
import matplotlib.pyplot as plt
dt = 100
for t in range(100):
    anim.animate(sim, dt)
    plt.pause(0.05)
plt.show()